In [1]:
from time import time
import pandas as pd
import numpy as np
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Import and clean relevant data

In [3]:
meta = pd.read_csv("../data/LINCS_DILI_metadata.csv")
GEXP_data = pd.read_csv("../data/L1000_Data_uncollapsed_all_conditionsv2_new.csv")
#external_test1 = pd.read_csv("../data/external_ambiguous_smiles.csv")
external_test = pd.read_csv("../../Challenge_Data/myname_predictions_no1_TEMPLATE.txt")

In [4]:
meta['vDILIConcern'].unique()

array(['vMost-DILI-Concern', 'vLess-DILI-Concern',
       'Ambiguous DILI-concern', 'vNo-DILI-Concern'], dtype=object)

In [5]:
ambiguous = external_test['Compound.Name'].tolist()
#ambiguos_meta = meta[meta['vDILIConcern'].isin(['Ambiguous DILI-concern'])]
ambiguos_meta = meta.loc[meta['pert_iname'].isin(ambiguous)]
meta = meta.loc[~meta['pert_iname'].isin(ambiguous)]

In [6]:
print(len(ambiguos_meta['pert_iname'].unique().tolist()),ambiguos_meta.shape)

55 (1617, 31)


In [7]:
temp_list = []
for index,row in ambiguos_meta.iterrows():
    temp_list.append(str(row['cell_id'])+'_'+str(row['pert_time'])+'_'+str(row['pert_dose'])+'_'+str(row['pert_iname']))

ambiguos_meta['combined_name'] = temp_list

In [8]:
temp_list = []
for index,row in meta.iterrows():
    temp_list.append(str(row['cell_id'])+'_'+str(row['pert_time'])+'_'+str(row['pert_dose'])+'_'+str(row['pert_iname']))

meta['combined_name'] = temp_list

In [9]:
meta.head()

,Unnamed: 0,id,sig_id,pert_id,pert_iname,pert_type,cell_id,pert_dose,pert_dose_unit,pert_idose,...,AdjudicatedDILI,vDILIConcern,Greene_Annotation,Sakatis_Annotation,Xu_Annotation,Zhu_Annotation,SMILES,URL..accessed.on.1.21.2016.,VER_DISP..1.LTKB.BD..2.others.,combined_name
0,1,CPC004_A375_6H:BRD-A14966924-001-03-0:10,CPC004_A375_6H:BRD-A14966924-001-03-0:10,BRD-A14966924,alaproclate,trt_cp,A375,10.0,µM,10 µM,...,.,vMost-DILI-Concern,.,.,.,.,CC(C(=O)OC(C)(C)CC1=CC=C(C=C1)Cl)N,NaN,1,A375_6_10.0_alaproclate
1,2,CPC004_A375_6H:BRD-K79131256-001-08-8:10,CPC004_A375_6H:BRD-K79131256-001-08-8:10,BRD-K79131256,albendazole,trt_cp,A375,10.0,µM,10 µM,...,Yes,vMost-DILI-Concern,HH,.,Positive,Postive,CCCSC1=CC2=C(C=C1)N=C(N2)NC(=O)OC,http://dailymed.nlm.nih.gov/dailymed/drugInfo....,2,A375_6_10.0_albendazole
2,3,CPC004_A375_6H:BRD-A09056319-001-05-5:10,CPC004_A375_6H:BRD-A09056319-001-05-5:10,BRD-A09056319,alfuzosin,trt_cp,A375,10.0,µM,10 µM,...,Yes,vLess-DILI-Concern,.,.,.,.,CN(CCCNC(=O)C1CCCO1)C2=NC3=CC(=C(C=C3C(=N2)N)O...,http://dailymed.nlm.nih.gov/dailymed/drugInfo....,2,A375_6_10.0_alfuzosin
3,4,CPC004_A375_6H:BRD-K30020243-051-01-7:10,CPC004_A375_6H:BRD-K30020243-051-01-7:10,BRD-K30020243,aliskiren,trt_cp,A375,10.0,µM,10 µM,...,Yes,vLess-DILI-Concern,.,.,.,.,CC(C)[C@@H](CC1=CC(=C(C=C1)OC)OCCCOC)C[C@@H]([...,http://dailymed.nlm.nih.gov/dailymed/drugInfo....,2,A375_6_10.0_aliskiren
4,5,CPC004_A375_6H:BRD-K17561142-003-16-8:10,CPC004_A375_6H:BRD-K17561142-003-16-8:10,BRD-K17561142,amiodarone,trt_cp,A375,10.0,µM,10 µM,...,Yes,vMost-DILI-Concern,HH,Hepatotoxic,Positive,Postive,CCCCC1=C(C2=CC=CC=C2O1)C(=O)C3=CC(=C(C(=C3)I)O...,http://dailymed.nlm.nih.gov/dailymed/drugInfo....,1,A375_6_10.0_amiodarone


In [10]:
len(list(set(meta['pert_iname'].unique().tolist()) & set(ambiguos_meta['pert_iname'].unique().tolist())))

0

In [11]:
# Split into seperate 'experiment' dataframes
dict_of_dfs = {k: v for k, v in meta.groupby(['cell_id','pert_time','pert_dose'])} # https://datascience.stackexchange.com/questions/29825/create-new-data-frames-from-existing-data-frame-based-on-unique-column-values
ambiguous_dict_of_dfs = {k: v for k, v in ambiguos_meta.groupby(['cell_id','pert_time','pert_dose'])}

In [12]:
dict_of_dfs['All_Cell_Lines'] = meta
ambiguous_dict_of_dfs['All_Cell_Lines'] = ambiguos_meta

In [18]:
dict_of_best_params = {}

for dic in dict_of_dfs:
    
    print(dic)
    dict_of_dfs[dic]
    id_list = dict_of_dfs[dic]['id'].tolist()
    pert_meta = dict_of_dfs[dic]['combined_name'].tolist() # meta to discren replicates
    pert_name = dict_of_dfs[dic]['pert_iname'].tolist()
    DILI_label = dict_of_dfs[dic]['vDILIConcern'].tolist() # output label
    data = GEXP_data[GEXP_data.columns & id_list]
    #data['Gene_ID'] = GEXP_data['id']
    data = data.T # cell-time-dose specific data
    #data.columns = id_list
    
    data.columns = GEXP_data['id'].tolist() # Add Gene ID's
    data['pert_meta'] = pert_meta
    data['pert_name'] = pert_name
    data['vDILIConcern'] = DILI_label
    data.reset_index(inplace=True)
    print(data.shape,len(data['pert_name'].unique().tolist()))
    
    print("----------------number of datapoints per DILI class----------------")
    print(data['vDILIConcern'].value_counts())
    print("----------------number of compounds per DILI class----------------")
    print(data.groupby('vDILIConcern')['pert_name'].nunique())
    
    if not os.path.exists('../data/Cell_line_Training_Data'):
        os.makedirs('../data/Cell_line_Training_Data')
    data.to_csv("../data/Cell_line_Training_Data/"+str(dic)+".csv")

('A375', 6, 10.0)
(478, 982) 178
----------------number of datapoints per DILI class----------------
vLess-DILI-Concern    240
vNo-DILI-Concern      144
vMost-DILI-Concern     94
Name: vDILIConcern, dtype: int64
----------------number of compounds per DILI class----------------
vDILIConcern
vLess-DILI-Concern    90
vMost-DILI-Concern    37
vNo-DILI-Concern      51
Name: pert_name, dtype: int64
('A549', 24, 10.0)
(364, 982) 178
----------------number of datapoints per DILI class----------------
vLess-DILI-Concern    172
vNo-DILI-Concern      129
vMost-DILI-Concern     63
Name: vDILIConcern, dtype: int64
----------------number of compounds per DILI class----------------
vDILIConcern
vLess-DILI-Concern    90
vMost-DILI-Concern    37
vNo-DILI-Concern      51
Name: pert_name, dtype: int64
('ASC', 24, 10.0)
(226, 982) 178
----------------number of datapoints per DILI class----------------
vLess-DILI-Concern    109
vNo-DILI-Concern       70
vMost-DILI-Concern     47
Name: vDILIConcern, dtype:

(2) External ambiguos compounds

In [20]:
for dic in dict_of_dfs:
    print(dic)
    ambiguous_meta = ambiguous_dict_of_dfs[dic][ambiguous_dict_of_dfs[dic]['pert_iname'].isin(ambiguous)]

    pert_name = ambiguous_meta['pert_iname'].tolist() # meta to discren replicates
    pert_meta = ambiguous_meta['combined_name'].tolist() # meta to discren replicates

    id_list = ambiguous_meta['id'].tolist()
    data = GEXP_data[GEXP_data.columns & id_list]
    data = data.T # cell-time-dose specific data
    data.columns = GEXP_data['id'].tolist() # Add Gene ID's
    data['pert_meta'] = pert_meta
    data['pert_name'] = pert_name
    #print(data)
    print(data.shape,len(data['pert_name'].unique().tolist()))
    

    
    if not os.path.exists('../data/Cell_line_External_Data'):
        os.makedirs('../data/Cell_line_External_Data')
    data.to_csv("../data/Cell_line_External_Data/External_"+str(dic)+".csv")

('A375', 6, 10.0)
(133, 980) 55
----------------number of compounds per DILI class----------------


KeyError: 'vDILIConcern'